In [37]:
from superdocs_python.utils.diff_utils import parse_diff
from datasets import load_dataset, IterableDataset, concatenate_datasets
from dotenv import load_dotenv
import os
import json
from superdocs_python.utils.model import create_model
import tiktoken
from datasets import Dataset
from functools import partial

# the prompt for generating is "Suggest rewrites that..."
#
# The output should first be a brief plan and then an execution based on the results
#

DELIM = "-----"

load_dotenv("../../.env")



False

In [38]:
import difflib
import re

def process_row(row):
    udiff = "\n".join(difflib.unified_diff(row['old_contents'].splitlines(), row['new_contents'].splitlines(), fromfile=row['old_file'], tofile=row['new_file'], n=3))
    udiff = f"```diff\n{udiff}\n```"
    udiff = re.sub("@@.*@@", "@@...@@", udiff)

    query = f"[{row['old_file']}]\n```\n{row['old_contents']}```\nGenerate a unified diff patch that completes the following task: \n{row['subject']}"
    
    return {"input": query, "output": udiff}


In [39]:
languages = ["python", "jsx", "kotlin", "vue", "xml", "javascript", "java", "c#", "ruby", "c", "c++", "typescript", "clojure", "dart"]
generated_datasets = []
for language in languages:
    dataset = load_dataset("bigcode/commitpackft", language, split="train")
    dataset = dataset.to_iterable_dataset()
    generated_datasets.append(dataset.map(lambda x: process_row(x)))
    
    
    

Using the latest cached version of the module from /Users/vijaydaita/.cache/huggingface/modules/datasets_modules/datasets/bigcode--commitpackft/a50f4045a7cb465d763f8230a42419d92af82798b829852b16383b225a647b03 (last modified on Mon Mar 25 14:23:59 2024) since it couldn't be found locally at bigcode/commitpackft, or remotely on the Hugging Face Hub.
Generating train split: 56025 examples [00:02, 22002.77 examples/s]
Generating train split: 100%|██████████| 9337/9337 [00:00<00:00, 272525.71 examples/s]


In [40]:
from datasets import DatasetDict

generated_regular_datasets = []
for generated_dataset in generated_datasets:
    ds = Dataset.from_generator(lambda: (yield from generated_dataset), features=generated_dataset.features)
    generated_regular_datasets.append(ds)

Generating train split: 56025 examples [00:06, 9281.80 examples/s] 
Generating train split: 2199 examples [00:00, 8583.81 examples/s]
Generating train split: 2214 examples [00:00, 8732.56 examples/s]
Generating train split: 587 examples [00:00, 8356.80 examples/s]
Generating train split: 9337 examples [00:00, 9528.58 examples/s] 
Generating train split: 52989 examples [00:06, 8570.74 examples/s]
Generating train split: 20635 examples [00:02, 8749.29 examples/s]
Generating train split: 9346 examples [00:01, 7789.05 examples/s]
Generating train split: 69413 examples [00:07, 8813.88 examples/s]
Generating train split: 8506 examples [00:00, 9176.22 examples/s]
Generating train split: 4992 examples [00:00, 8984.54 examples/s]
Generating train split: 5868 examples [00:00, 8260.53 examples/s]
Generating train split: 2403 examples [00:00, 9711.29 examples/s]
Generating train split: 765 examples [00:00, 9125.42 examples/s]


In [42]:
ds = concatenate_datasets(generated_regular_datasets)
ds.push_to_hub("vdaita/commitpackft-patches")




Uploading the dataset shards: 100%|██████████| 2/2 [00:19<00:00,  9.76s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/vdaita/commitpackft-patches/commit/d6284afb450ed1288851f0cde3eb9a8426023416', commit_message='Upload dataset', commit_description='', oid='d6284afb450ed1288851f0cde3eb9a8426023416', pr_url=None, pr_revision=None, pr_num=None)